# Stable Baselines3 Tutorial - Getting Started

Stable-Baselines3: https://github.com/DLR-RM/stable-baselines3

Documentation: https://stable-baselines3.readthedocs.io/en/master/

RL Baselines3 zoo: https://github.com/DLR-RM/rl-baselines3-zoo

[RL Baselines3 Zoo](https://github.com/DLR-RM/rl-baselines3-zoo) :Stable-Baselines3을 이용해서 pre-trained Reinforcement Learning agents 를 제공합니다.
또한 기본적인 training scripts, evaluating agents, tuning hyperparameters, recording videos도 제공합니다.



## Introduction

1. Stable baselines library을 이용해서 간편하게 RL model을 만들고, train 과 evaluate 하는 법을 배워봅시다.
2. 모든 알고리즘이 같은 인터페이스를 이용하기 때문에, 아래의 예제를 여러가지 다양한 알고리즘에 적용해서 실험해 볼 수 있습니다.
3. 이 튜토리얼은 open ai gym 환경에서만 돌아가므로, 우리가 원하는 환경을 open ai gym 형태에 맞추어서 customized environment 를 만드는 것이 필요합니다. 


## Install Dependencies and Stable Baselines3 

List of full dependencies :[README](https://github.com/DLR-RM/stable-baselines3).


```conda install -c conda-forge stable-baselines3```

 ## Quick and Dirty Tutorial

Stable-Baselines3 은 [gym interface] 이용합니다.(https://stable-baselines3.readthedocs.io/en/master/guide/custom_env.html).
- available environment 의 리스트는 여기에 있습니다. [here](https://gym.openai.com/envs/#classic_control).
- It is also recommended to check the [source code](https://github.com/openai/gym) to learn more about the observation and action space of each env, as gym does not have a proper documentation.
- [recap table](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html)

In [9]:
#import modules
import gym
from stable_baselines3 import DQN

In [10]:
#make environment
env = gym.make("CartPole-v0")

In [11]:
#make module
model = DQN("MlpPolicy", env, verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [14]:
#Train DQN model for 10000 timesptes
model.learn(total_timesteps=10000, log_interval=100)

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 21.3     |
|    ep_rew_mean      | 21.3     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 100      |
|    fps              | 11862    |
|    time_elapsed     | 0        |
|    total_timesteps  | 2126     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 23       |
|    ep_rew_mean      | 23       |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 200      |
|    fps              | 12284    |
|    time_elapsed     | 0        |
|    total_timesteps  | 4426     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 20       |
|    ep_rew_mean      | 20       |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes       

In [15]:
#save model
model.save("dqn_cartpole")

In [16]:
#load model
del model # remove to demonstrate saving and loading
model = DQN.load("dqn_cartpole")

In [19]:
#Let's see how the model works
obs = env.reset()
for i in range(1000):
    # predict from loaded model
    action, _states = model.predict(obs, deterministic=True) 
    obs, reward, done, info = env.step(action)
    env.render()
    if done:
      obs = env.reset()

## 참 쉽지요~?

## 그럼 하나하나 다시 뜯어서 자세히 보아요

## First, start with importing modules

In [20]:
import gym
import numpy as np

제일먼저 해야할 일은 RL model 을 import 하는것 입니다.
Available 한 모델이 무엇인지 보시려면 documentation 을 확인하시면 됩니다. : A2C, DDPG, PPO, DQN, SAC, HER, TD3

In [21]:
from stable_baselines3 import PPO

그 다음으로는 the policy/value functions 을 만들 network 를 import 하는 것 입니다.
이 스텝은 optional 이고, 지정을 안해줬을때는 the constructor에서 기본으로 지정된데로 (원본 논문에서 사용한 네트웍을 이용해서) 생성됩니다. 

```PPO('MlpPolicy', env)``` instead of ```PPO(MlpPolicy, env)```

Note that some algorithms like `SAC` have their own `MlpPolicy`--> 이럴 경우에는 인위적으로 바꾸어주지 않고 기본값을 사용합니다.

In [22]:
from stable_baselines3.ppo.policies import MlpPolicy

## Create the Gym env and instantiate the agent

PPO 를 이용하여 CartPole environment를 트레인 해봅시다.

CartPole environment의 Control Problem 은 다음과 같습니다. 
"pole 이 un-actuated joint 를 이용해서 cart에 부착되어 있음.
 cart 가 frictionless track을 움직임.
 The system 이  +1 (오른쪽) or -1(왼쪽)방향으로 cart를 밀면서 cart 위 부착된 pendulum 이 falling over 되는것을 막는것이 목표이다. 
 A reward of +1 is provided for every timestep that the pole remains upright. "

Cartpole environment: [https://gym.openai.com/envs/CartPole-v1/](https://gym.openai.com/envs/CartPole-v1/)

![Cartpole](https://cdn-images-1.medium.com/max/1143/1*h4WTQNVIsvMXJTCpXm_TAw.gif)


1. 여기서 우리는 MlpPolicy를 Policy Network로 사용합니다.
   CNN 대신 DNN 을 쓰는 이유는 observation of the CartPole task가  feature vector 이기 때문입니다. ( not images. )
2. RL algorithm: [Proximal Policy Optimization](https://stable-baselines3.readthedocs.io/en/master/modules/ppo2.html) algorithm:
 Actor-Critic method 의 하나로  value function 을 사용하여 policy gradient descent 를 수행합니다. 
3. PPO 는 on-policy algorithm 입니다. : policy network 을 업데이트 하기위해 사용되는 trajectories 가 항상 latest policy 에 의해 샘플 됩니다.
   Sample effeciency 측면에서 off-policy alorithms([DQN](https://stable-baselines.readthedocs.io/en/master/modules/dqn.html), [SAC](https://stable-baselines3.readthedocs.io/en/master/modules/sac.html) or [TD3](https://stable-baselines3.readthedocs.io/en/master/modules/td3.html)) 보다는 "less sample efficient"하지만 wall-clock time 측면에서는  "much faster" 합니다.


In [23]:
env = gym.make('CartPole-v1')

model = PPO(MlpPolicy, env, verbose=0)

Agent 를 evaluate 하기 위해, 트레인된 모델을 이용하여, 에피소드를 몇번 실행하고 나오는 reward를 average하는 evaluate function 을 만들어 봅시다

In [24]:
def evaluate(model, num_episodes=100):
    """
    Evaluate a RL agent
    :param model: (BaseRLModel object) the RL Agent
    :param num_episodes: (int) number of episodes to evaluate it
    :return: (float) Mean reward for the last num_episodes
    """
    # This function will only work for a single Environment
    env = model.get_env()
    all_episode_rewards = []
    for i in range(num_episodes):
        episode_rewards = []
        done = False
        obs = env.reset()
        while not done:
            # _states are only useful when using LSTM policies
            action, _states = model.predict(obs)
            # here, action, rewards and dones are arrays
            # because we are using vectorized env
            obs, reward, done, info = env.step(action)
            episode_rewards.append(reward)

        all_episode_rewards.append(sum(episode_rewards))

    mean_episode_reward = np.mean(all_episode_rewards)
    print("Mean reward:", mean_episode_reward, "Num episodes:", num_episodes)

    return mean_episode_reward

자! 이제 un-trained agent(random agent) 를 이용하여 정확도를 측정해보지요!

In [25]:
# Random Agent, before training
mean_reward_before_train = evaluate(model, num_episodes=100)

Mean reward: 21.93 Num episodes: 100


Stable-Baselines API도 트레인된 모델을 evaluation 하는  helper function 을 제공하지요:

In [26]:
from stable_baselines3.common.evaluation import evaluate_policy

In [27]:
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

/Users/sookim/miniconda3/envs/aiai2022-rl/lib/python3.8/site-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


mean_reward:9.52 +/- 0.61


## Train the agent and evaluate it

자! 그럼 모델을 트레인 해보고 랜덤일 경우에 비해 얼마나 좋아지나 볼까요?

In [28]:
# Train the agent for 10000 steps
model.learn(total_timesteps=10000)

In [29]:
# Evaluate the trained agent
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

mean_reward:317.03 +/- 124.28


예상대로 training 이 잘되었고 the mean reward 가 많이 증가했네요!! yeah!

### Prepare video recording

모델을 잘 트레인 한걸 확인했으니, 이제 트레인된 모델을 가져와서 한번 에피소드를 실행해 보고 그걸 비디오로 녹화해 보아요.

In [37]:
# Set up fake display; otherwise rendering will fail
import os
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

sh: Xvfb: command not found


In [39]:
import base64
from pathlib import Path

from IPython import display as ipythondisplay

In [32]:
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

def record_video(env_id, model, video_length=500, prefix='', video_folder='videos/'):
  """
  :param env_id: (str)
  :param model: (RL model)
  :param video_length: (int)
  :param prefix: (str)
  :param video_folder: (str)
  """
  eval_env = DummyVecEnv([lambda: gym.make(env_id)])
  # Start the video at step=0 and record 500 steps
  eval_env = VecVideoRecorder(eval_env, video_folder=video_folder,
                              record_video_trigger=lambda step: step == 0, video_length=video_length,
                              name_prefix=prefix)

  obs = eval_env.reset()
  for _ in range(video_length):
    action, _ = model.predict(obs)
    obs, _, _, _ = eval_env.step(action)

  # Close the video recorder
  eval_env.close()

### Visualize trained agent



In [40]:
record_video('CartPole-v1', model, video_length=500, prefix='ppo2-cartpole')

Saving video to /Users/sookim/lecture/RL_toturial_AIAI2022/videos/ppo2-cartpole-step-0-to-step-500.mp4


In [41]:
from IPython.display import Video
Video("/Users/sookim/lecture/RL_toturial_AIAI2022/videos/ppo2-cartpole-step-0-to-step-500.mp4")

## Bonus: Train a RL Model in One Line

아래 코멘드를 이용하여 한 줄로 모델을 간단히 트레인 할수도 있습니다.
단 아래와 같은 한줄 코멘드는 레지스터드 된 모델에 한해서만 가능하지요[registered](https://stable-baselines3.readthedocs.io/en/master/guide/quickstart.html).

In [43]:
model = PPO('MlpPolicy', "CartPole-v1", verbose=1).learn(1000)

Using cpu device
Creating environment from the given name 'CartPole-v1'
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 24.1     |
|    ep_rew_mean     | 24.1     |
| time/              |          |
|    fps             | 2470     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------


## Conclusion

이번 튜토리얼에서는 s- how to define and train a RL model using stable baselines3, it takes only one line of code ;)